In [24]:
import pandas as pd
import os
import re
import warnings
from statsbombpy import sb
import numpy as np

warnings.filterwarnings("ignore", category=UserWarning)
comps = sb.competitions()

In [ ]:
# Frank Rijkaard (2003–2008)         - season id: 37, 38, 39, 40
# Pep Guardiola (2008–2012)          - season id: 41, 21, 22, 23
# Tito Vilanova (2012–2013)          - season id: 24
# Gerardo "Tata" Martino (2013–2014) - season id: 25
# Luis Enrique (2014–2017)           - season id: 26, 27, 2
# Ernesto Valverde (2017–2020)       - season id: 1, 4, 42 (303652-ig)
# Quique Setién (2020)               - season id: 42 (303652 utan)
# Ronald Koeman (2020–2021)          - season id: 90

In [25]:
# 1. Frank Rijkaard (2003–2008)         - season id: 37, 38, 39, 40
fr_s1 = sb.matches(competition_id=11, season_id=37)
fr_s2 = sb.matches(competition_id=11, season_id=38)
fr_s3 = sb.matches(competition_id=11, season_id=39)
fr_s4 = sb.matches(competition_id=11, season_id=40)
matches_under_rijkaard = pd.concat([fr_s1, fr_s2, fr_s3, fr_s4], ignore_index=True)

In [26]:
# 2. Pep Guardiola (2008–2012) - season id: 41, 21, 22, 23
pg_s1 = sb.matches(competition_id=11, season_id=41)
pg_s2 = sb.matches(competition_id=11, season_id=21)
pg_s3 = sb.matches(competition_id=11, season_id=22)
pg_s4 = sb.matches(competition_id=11, season_id=23)
matches_under_guardiola = pd.concat([pg_s1, pg_s2, pg_s3, pg_s4], ignore_index=True)

In [27]:
# 3. Tito Vilanova (2012–2013) - season id: 24
tv_s1 = sb.matches(competition_id=11, season_id=24)
matches_under_vilanova = tv_s1

In [28]:
# 4. Gerardo "Tata" Martino (2013–2014) - season id: 25
gm_s1 = sb.matches(competition_id=11, season_id=25)
matches_under_martino = gm_s1

In [29]:
# 5. Luis Enrique (2014–2017) - season id: 26, 27, 2
le_s1 = sb.matches(competition_id=11, season_id=26)
le_s2 = sb.matches(competition_id=11, season_id=27)
le_s3 = sb.matches(competition_id=11, season_id=2)
matches_under_enrique = pd.concat([le_s1, le_s2, le_s3], ignore_index=True)

In [30]:
# 6. Ernesto Valverde (2017–2020) - season id: 1, 4, 42 (303652-ig)
ev_s1 = sb.matches(competition_id=11, season_id=1)
ev_s2 = sb.matches(competition_id=11, season_id=4)
ev_s3_full = sb.matches(competition_id=11, season_id=42)
ev_s3 = ev_s3_full[ev_s3_full['match_id'] <= 303652]
matches_under_valverde = pd.concat([ev_s1, ev_s2, ev_s3], ignore_index=True)

In [31]:
# 7. Quique Setién (2020) - season id: 42 (303652 után)
qs_s1_full = sb.matches(competition_id=11, season_id=42)
qs_s1 = qs_s1_full[qs_s1_full['match_id'] > 303652]
matches_under_setien = qs_s1

In [32]:
# 8. Ronald Koeman (2020–2021) - season id: 90
rk_s1 = sb.matches(competition_id=11, season_id=90)
matches_under_koeman = rk_s1

In [33]:
eras = [matches_under_rijkaard, matches_under_guardiola, matches_under_vilanova, matches_under_martino, matches_under_enrique, matches_under_valverde, matches_under_setien, matches_under_koeman]

In [ ]:
team = "Barcelona"
i = 0
for matches in eras:
    i=i+1
    team_matches = matches[(matches['home_team'] == team) | (matches['away_team'] == team)].sort_values(by='match_date')
    team_match_ids = team_matches['match_id'].tolist()
    print(len(team_match_ids))

    safe_team = re.sub(r'[^\w\s]', '', team).replace(" ", "_")
    output_dir = f"BarcelonaCoaches/{i}"
    os.makedirs(output_dir)
    
    season_stats = []
    
    for match_id in team_match_ids:
        match = team_matches[team_matches['match_id'] == match_id]
        events = sb.events(match_id=match_id)
        print("")
        print(match_id, match['match_date'].values[0], match['home_team'].values[0], match['away_team'].values[0], ":")
        lineup = sb.lineups(match_id=match_id)
        player_ids = lineup[team].player_id.to_list()
        player_names = lineup[team].player_name.to_list()
        player_list = list(zip(player_ids, player_names))

        stating_xi_events = events[events['type'] == "Starting XI"]
        lineup_team = stating_xi_events[stating_xi_events['team']  == team]
        new_df = lineup_team.copy()
        new_df['lineup'] = new_df['tactics'].apply(lambda x: x.get('lineup') if isinstance(x, dict) else None)
    
        match_stats = []
        
        print("")
        for player_id, player_name in player_list:
            
            events_player = events[events['player_id'] == player_id ]
            
            # 0. :
            
            new_df['is_in_starting_xi'] = new_df['lineup'].apply(
                lambda lineup: any(player['player']['id'] == player_id for player in lineup) if isinstance(lineup, list) else False
            )
            new_df = new_df.reset_index(drop=True)
            is_starting_ = new_df.loc[0, 'is_in_starting_xi']
            is_starting = bool(is_starting_)
            is_starting
                    
            substitution_events = events[events['type'] == "Substitution"]
            player_start_time = 0
            player_end_time = 90
            if not is_starting:
                player_start_time = 90
            sub_in_event = substitution_events[(substitution_events['substitution_replacement_id'] == player_id)]
            sub_out_event = substitution_events[(substitution_events['player_id'] == player_id)]
            if not sub_in_event.empty:
                player_start_time = sub_in_event['minute'].iloc[0]  # Get the minute of substitution
            if not sub_out_event.empty:
                player_end_time = sub_out_event['minute'].iloc[0]  # Get the minute of substitution
            minutes_played_ = player_end_time - player_start_time
            minutes_played = int(minutes_played_)
    
            if minutes_played <= 0:
                continue
    
            print(player_id, player_name)
            print("")
            
            #print("Jatszott percek:", minutes_played)
    
            player_stats = {
                'player_id': player_id,
                'player_name': player_name,
                'minutes_played': minutes_played,
            }
    
            shot_events = events_player[events_player['type'] == "Shot"]
    
            # 1. :
            shot_count = shot_events.shape[0]
            player_stats['shot_count'] = shot_count
            #print("Lovesek szama:", shot_count)
    
            # 2. :
            goal_count = shot_events[shot_events['shot_outcome'] == "Goal"].shape[0]
            player_stats['goal_count'] = goal_count
            #print("Golok szama:", goal_count)
    
            # 3. :
            xG_ = shot_events[["shot_statsbomb_xg"]].sum()
            xG = float(xG_.iloc[0])
            player_stats['xG'] = xG
            #print("Termelt xG:", xG)
    
            # 4. :
            on_target = shot_events[shot_events['shot_outcome'].isin(["Goal", "Saved", "Saved To Post"])].shape[0]
            player_stats['on_target'] = on_target
            #print("Kapura tarto lovesek szama:", on_target)

            pass_events = events_player[events_player['type'] == "Pass"]
    
            # 5. :
            pass_count = pass_events.shape[0]
            player_stats['pass_count'] = pass_count
            #print("Passzok száma:", pass_count)
    
            # 6. :
            wrong_passes = pass_events[pass_events['pass_outcome'].isin(["Incomplete", "Out", "Pass Offside"])].shape[0]
            successfull_passes = pass_count - wrong_passes
            player_stats['successfull_pass_count'] = successfull_passes
            #print("Passzpontosság:", successfull_passes)
    
            # 7. :
            assists = 0
            if 'pass_goal_assist' in pass_events.columns:
                assists = pass_events[pass_events['pass_goal_assist'] == True].shape[0]
            player_stats['assists'] = assists
            #print("Gólpasszok száma:", assists)
    
            # 8. :
            shot_assists = pass_events[pass_events['pass_shot_assist'] == True].shape[0]
            key_passes = assists + shot_assists
            player_stats['key_passes'] = key_passes
            #print("Kulcspasszok száma:", key_passes)
    
            # 9. :
            pass_assisted_shot_id = pass_events[pass_events['pass_assisted_shot_id'].notna()]
            pass_assisted_shot_id[['pass_assisted_shot_id']]
            shot_ids = pass_assisted_shot_id['pass_assisted_shot_id'].tolist()
            ee = events[events['id'].isin(shot_ids)]
            xA_ = ee[["shot_statsbomb_xg"]].sum()
            xA = float(xA_.iloc[0])
            player_stats['xA'] = xA
            #print("Termelt xA:", xA)
    
            dribble_events = events_player[events_player['type'] == "Dribble"]
    
            # 12. :
            dribble_count = dribble_events.shape[0]
            player_stats['dribble_count'] = dribble_count
            #print("Cselek száma:", dribble_count)
    
            # 13. :
            successfull_dribbles = dribble_events[dribble_events['dribble_outcome'] == "Complete"].shape[0]
            player_stats['successfull_dribbles'] = successfull_dribbles
            #print("Sikeres cselek száma:", successfull_dribbles)
    
            carry_events = events_player[events_player['type'] == "Carry"]
    
            # 14. :
            carry_count = carry_events
            carry_count = carry_count.copy()
            carry_count['carry_length'] = carry_count.apply(
                lambda row: np.linalg.norm(np.array(row['carry_end_location']) - np.array(row['location'])), axis=1
            )
            carries = carry_count[['location', 'carry_end_location', 'carry_length']]
            long_carries = carries[carries['carry_length'] >= 10]
            player_stats['long_carries'] = long_carries.shape[0]
            #print("Labdavezetések száma:", long_carries.shape[0])
    
            foul_committed_events = events_player[events_player['type'] == "Foul Committed"]
    
            # 15. :
            foul_committed_count = foul_committed_events.shape[0]
            player_stats['foul_committed_count'] = foul_committed_count
            #print("Elkövetett szabálytalanságok száma:", foul_committed_count)
    
            foul_won_events = events_player[events_player['type'] == "Foul Won"] 
    
            # 16. :
            fouls_suffered_events = events_player[events_player['type'] == "Foul Won"]
            fouls_suffered_count = fouls_suffered_events.shape[0]
            player_stats['fouls_suffered_count'] = fouls_suffered_count
            #print("Elszenvedett szabálytalanságok száma:", fouls_suffered_count)
    
            # 17. :
            penalites_won = 0
            if foul_won_events.shape[0] and 'foul_won_defensive' in foul_won_events.columns :
                penalites_won = foul_won_events[foul_won_events['foul_won_defensive'] == True].shape[0]
            player_stats['penalites_won'] = penalites_won
            #print("Védekezés közben elszenvedett szabálytalanságok száma:", penalites_won)
    
            duel_events = events_player[events_player['type'] == "Duel"]
    
            # 18. :
            duel_count = duel_events.shape[0]
            player_stats['duel_count'] = duel_count
            #print("Párharcok száma:", duel_count)
            
            # 19. :
            won_duels = duel_events[duel_events['duel_outcome'].isin(["Won", "Success", "Success In Play", "Success Out"])].shape[0]
            player_stats['won_duels'] = won_duels
            #print("Sikeres párharcok száma:", won_duels)
    
            # 20. :
            ball_recovery_events = events_player[events_player['type'] == "Ball Recovery"]
            ball_recovery_count = ball_recovery_events.shape[0]
            player_stats['ball_recovery_count'] = ball_recovery_count
            print("Labda visszaszerzések száma:", ball_recovery_count)
    
            # 21. :
            dispossessed_events = events_player[events_player['type'] == "Dispossessed"]
            dispossessed_count = dispossessed_events.shape[0]
            player_stats['dispossessed_count'] = dispossessed_count
            #print("Labdavesztések száma:", dispossessed_count)
    
            # 22. :
            clearance_events = events_player[events_player['type'] == "Clearance"]
            clearance_count = clearance_events.shape[0]
            player_stats['clearance_count'] = clearance_count
            #print("Tisztázások száma:", clearance_count)
    
            # 23. :
            interception_events = events_player[events_player['type'] == "Interception"]
            interception_count = interception_events.shape[0]
            player_stats['interception_count'] = interception_count
            #print("Elcsípett passzok száma:", interception_count)
    
            # 24. :
            block_events = events_player[events_player['type'] == "Block"]
            block_count = block_events.shape[0]
            player_stats['block_count'] = block_count
            #print("Blokkolások száma:", block_count)
    
            # 25. :
            tackle_events = events_player[(events_player['type'] == "Duel") & (events_player['duel_type'] == "Tackle")]
            tackle_count = tackle_events.shape[0]
            player_stats['tackle_count'] = tackle_count
            #print("Szerelések száma:", tackle_count)
    
            # 26. :
            successful_tackles = tackle_events[tackle_events['duel_outcome'] == "Won"].shape[0]
            player_stats['successful_tackles'] = successful_tackles
            #print("Sikeres szerelések száma:", successful_tackles)
    
            # 27. :
            pressure_events = events_player[events_player['type'] == "Pressure"]
            pressure_count = pressure_events.shape[0]
            player_stats['pressure_count'] = pressure_count
            #print("Nyomás alá helyezések (pressing) száma:", pressure_count)
    
            print("---------------------------------")   
    
            match_stats.append(player_stats)
    
        df_match_stats = pd.DataFrame(match_stats)
        match_date_str = match['match_date'].values[0].replace("-", "")
        match_filename = f"{output_dir}/match_{match_id}_{match_date_str}.csv"
        df_match_stats.to_csv(match_filename, index=False)
    
        season_stats.extend(match_stats)
    
    df_season_stats = pd.DataFrame(season_stats)
    
    season_aggregate_stats = df_season_stats.groupby(['player_id', 'player_name'], as_index=False).sum(numeric_only=True)

    season_filename = f"{output_dir}/{i}_coach_stats.csv"
    season_aggregate_stats.to_csv(season_filename, index=False)

78

68353 2004-10-16 Espanyol Barcelona :

5216 Andrés Iniesta Luján

Jatszott percek: 24

Jatszott percek: 24
Lovesek szama: 0
Golok szama: 0
Termelt xG: 0.0
Kapura tarto lovesek szama: 0
Passzok száma: 18
Passzpontosság: 14
Gólpasszok száma: 0
Kulcspasszok száma: 0
Termelt xA: 0.0
Cselek száma: 1
Sikeres cselek száma: 1
Labdavezetések száma: 3
Elkövetett szabálytalanságok száma: 0
Elszenvedett szabálytalanságok száma: 2
Védekezés közben elszenvedett szabálytalanságok száma: 1
Párharcok száma: 2
Sikeres párharcok száma: 1
Labda visszaszerzések száma: 3
Labdavesztések száma: 0
Tisztázások száma: 0
Elcsípett passzok száma: 0
Blokkolások száma: 0
Szerelések száma: 1
Sikeres szerelések száma: 1
Nyomás alá helyezések (pressing) száma: 4
---------------------------------
5503 Lionel Andrés Messi Cuccittini

Jatszott percek: 9

Jatszott percek: 9
Lovesek szama: 0
Golok szama: 0
Termelt xG: 0.0
Kapura tarto lovesek szama: 0
Passzok száma: 2
Passzpontosság: 1
Gólpasszok száma: 0
Kulcspasszok s